In [ ]:
import scipy.io
import numpy as np

# Load the train.mat file
mat_file_path = "train.mat"
data = scipy.io.loadmat(mat_file_path)

# Display the keys in the loaded data
data.keys()


In [ ]:
# Extract the Train_data object
train_data = data['Train_data']

# Check the type and shape
type(train_data), train_data.shape


In [ ]:
train_data[0].size

In [ ]:
train_data.shape[1]

In [ ]:
# Extract the first system (system 0)
system_0 = train_data[0, 0]

# Check what fields or attributes are available in this system's structure
system_0.dtype.names


In [ ]:
system_0

In [ ]:
# Extract trajectory and label for system 0
trajectory_0 = system_0['trajectory']
label_0 = system_0['Label']

# Check shapes of both arrays
trajectory_0.shape, label_0.shape


In [ ]:
trajectory_0[0,0].shape

In [ ]:
label_0[0].shape


In [ ]:
# Extract trajectory of component 0
component_0_traj = trajectory_0[0, 0]  # 1st row, 1st component

# Check shape and type
type(component_0_traj), component_0_traj.shape


In [ ]:
all_normal_data = []

for i in range(train_data.shape[1]):
    system = train_data[0, i]
    trajectories = system['trajectory']  # shape (1, 4)
    labels = system['Label']            # shape (4, 1000)
    
    for j in range(4):  # For each of the 4 trajectories per system
        traj = trajectories[0, j]       # shape (10, 1000)
        label = labels[j]               # shape (1000,)
        
        # Select only columns (time steps) where label is 0
        normal_indices = label == 0
        normal_data = traj[:, normal_indices]  # shape (10, N_normal)
        
        # Transpose to (N_normal, 10)
        normal_data = normal_data.T
        all_normal_data.append(normal_data)

# Stack all normal data
X_train = np.vstack(all_normal_data) 
print("Shape of normal trajectory: " , X_train.shape)

In [ ]:
# Collect all anomalous samples
all_anomaly_data = []

for i in range(train_data.shape[1]):
    system = train_data[0, i]
    trajectories = system['trajectory']  # shape (1, 4)
    labels = system['Label']            # shape (4, 1000)
    
    for j in range(4):  # For each of the 4 trajectories per system
        traj = trajectories[0, j]       # shape (10, 1000)
        label = labels[j]               # shape (1000,)
        
        # Select only columns (time steps) where label is 1
        anomaly_indices = label == 1
        anomaly_data = traj[:, anomaly_indices]  # shape (10, N_anomaly)
        
        # Transpose to (N_anomaly, 10)
        anomaly_data = anomaly_data.T
        all_anomaly_data.append(anomaly_data)

# Stack all anomaly data
X_anomaly = np.vstack(all_anomaly_data)  # Final shape: (N_total_anomaly, 10)
print("Anomaly data shape:", X_anomaly.shape)

In [ ]:
import pandas as pd
import torch
from torch import nn, optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device ", device)

In [ ]:
# Train/test split
train_df, test_df = train_test_split(
    X_train,  # Ensure we only use the specified features
    test_size=0.2,
    random_state=42
)

In [ ]:
# Scale data with standard scaler
#scaler = StandardScaler()
#train_df_scaled = scaler.fit_transform(train_df)
#test_df_scaled = scaler.transform(test_df)
#anomaly_scaled = scaler.transform(X_anomaly)  

# Scale data with min-max scaler
#scaler = MinMaxScaler()
scaler = MinMaxScaler(feature_range=(-1, 1))
train_df_scaled = scaler.fit_transform(train_df)
test_df_scaled = scaler.transform(test_df)
anomaly_scaled = scaler.transform(X_anomaly)  

In [ ]:
print("shape of train data ", train_df_scaled.shape)
print("shape of test data ", test_df_scaled.shape)
print("shape of anomaly data ", anomaly_scaled.shape)

In [ ]:
# Convert to PyTorch tensors
train_data_tensor = torch.FloatTensor(train_df_scaled).to(device)
test_data_tensor = torch.FloatTensor(test_df_scaled).to(device)
anomaly_data_tensor = torch.FloatTensor(anomaly_scaled).to(device)


# Step 3: GAN Architecture


data_dim = train_data_tensor.shape[1]
data_dim

In [ ]:
latent_dim =8  # for encoder part of the 
class Encoder(nn.Module):
    def __init__(self, data_dim, latent_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(data_dim, 64),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(64),
            nn.Linear(64, 32),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(32),
            nn.Linear(32, 16),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(16),
            nn.Linear(16, latent_dim),
            nn.LeakyReLU(0.2)
        )

    def forward(self, x):
        return self.model(x)

class Decoder(nn.Module):
    def __init__(self, data_dim, latent_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 16),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(16),
            nn.Linear(16, 32),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(32),
            nn.Linear(32, 64),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(64),
            nn.Linear(64, data_dim),
            nn.Tanh()  # since scaled to [-1, 1]
        )

    def forward(self, z):
        return self.model(z)


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, data_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(data_dim, 32),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(32, 16),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


In [ ]:
class Encoder2(nn.Module):  # Same structure as Encoder1
    def __init__(self, data_dim, latent_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(data_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128,64),
            nn.LeakyReLU(0.2),
            nn.Linear(64,32),
            nn.LeakyReLU(0.2),
            nn.Linear(32,latent_dim),
        )

    def forward(self, x):
        return self.model(x)


In [ ]:
# Step 4: Instantiate models
encoder = Encoder(data_dim, latent_dim).to(device)
decoder = Decoder(data_dim, latent_dim).to(device)
encoder2 = Encoder2(data_dim, latent_dim).to(device)

discriminator = Discriminator(data_dim).to(device)

loss_ad = nn.BCELoss()
#loss_rec = nn.MSELoss()
loss_rec = nn.L1Loss()
loss_enc = nn.MSELoss()


#lr = 0.0001

In [ ]:
#opt_g = optim.Adam(generator.parameters(), lr=lr)
#opt_d = optim.Adam(discriminator.parameters(), lr=lr)

opt_g = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=0.0002,betas=(0.5, 0.999))
opt_en2 = optim.Adam(encoder2.parameters(), lr=0.0002, betas=(0.5, 0.999) )
opt_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999) )


# Step 5: Training loop
epochs = 150
batch_size = 512

# Weights for loss terms
w_ad = 1
w_rec = 50
w_enc = 1  

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

dataset = TensorDataset(train_data_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
import torch
import os
from tqdm import tqdm

# === Setup for checkpointing ===
checkpoint_path = 'checkpoints/'
checkpoint_file = os.path.join(checkpoint_path, 'ganomaly_checkpoint.pth')
os.makedirs(checkpoint_path, exist_ok=True)

start_epoch = 0
best_loss_d = float('inf')

# === Try to load checkpoint if it exists ===
if os.path.exists(checkpoint_file):
    checkpoint = torch.load(checkpoint_file)
    encoder.load_state_dict(checkpoint['encoder_state_dict'])
    decoder.load_state_dict(checkpoint['decoder_state_dict'])
    discriminator.load_state_dict(checkpoint['discriminator_state_dict'])
    encoder2.load_state_dict(checkpoint['encoder2_state_dict'])

    opt_g.load_state_dict(checkpoint['opt_g_state_dict'])
    opt_d.load_state_dict(checkpoint['opt_d_state_dict'])
    opt_en2.load_state_dict(checkpoint['opt_en2_state_dict'])

    start_epoch = checkpoint['epoch'] + 1
    best_loss_d = checkpoint['best_loss_d']

    print(f"✅ Loaded checkpoint from epoch {start_epoch - 1}")

# === Lists to store losses ===
losses_d = []
losses_g = []
losses_rec = []
losses_enc = []

for epoch in tqdm(range(start_epoch, epochs)):
    epoch_loss_d = 0.0
    epoch_loss_g = 0.0
    epoch_loss_rec = 0.0
    epoch_loss_enc = 0.0
    num_batches = 0

    for batch_idx, (real_data,) in enumerate(dataloader):
        num_batches += 1
        real_data = real_data.to(device)

        # === Train Discriminator ===
        latent = encoder(real_data)
        fake_data = decoder(latent).detach()

        d_real = discriminator(real_data)
        d_fake = discriminator(fake_data)

        real_labels = torch.ones(real_data.size(0), 1, device=device)
        fake_labels = torch.zeros(fake_data.size(0), 1, device=device)

        loss_d_real = loss_ad(d_real, real_labels)
        loss_d_fake = loss_ad(d_fake, fake_labels)
        loss_d = (loss_d_real + loss_d_fake) / 2

        opt_d.zero_grad()
        loss_d.backward()
        opt_d.step()

        # === Train Generator (encoder + decoder) ===
        latent = encoder(real_data)
        fake_data = decoder(latent)
        d_fake = discriminator(fake_data)

        loss_ad_g = loss_ad(d_fake, real_labels)
        loss_rec_g = loss_rec(fake_data, real_data)
        loss_g = w_ad * loss_ad_g + w_rec * loss_rec_g

        opt_g.zero_grad()
        loss_g.backward()
        opt_g.step()

        # === Train Encoder2 ===
        with torch.no_grad():
            latent = encoder(real_data)
            fake_data = decoder(latent)

        latent_recon = encoder2(fake_data)
        loss_enc_g = loss_enc(latent, latent_recon) * w_enc

        opt_en2.zero_grad()
        loss_enc_g.backward()
        opt_en2.step()

        # === Accumulate losses ===
        epoch_loss_d += loss_d.item()
        epoch_loss_g += loss_g.item()
        epoch_loss_rec += loss_rec_g.item()
        epoch_loss_enc += loss_enc_g.item()

    # === Average Epoch Losses ===
    avg_loss_d = epoch_loss_d / num_batches
    avg_loss_g = epoch_loss_g / num_batches
    avg_loss_rec = epoch_loss_rec / num_batches
    avg_loss_enc = epoch_loss_enc / num_batches

    losses_d.append(avg_loss_d)
    losses_g.append(avg_loss_g)
    losses_rec.append(avg_loss_rec)
    losses_enc.append(avg_loss_enc)

    # === Save checkpoint every N epochs or on best loss improvement ===
    if epoch % 10 == 0:
        torch.save({
            'epoch': epoch,
            'encoder_state_dict': encoder.state_dict(),
            'decoder_state_dict': decoder.state_dict(),
            'discriminator_state_dict': discriminator.state_dict(),
            'encoder2_state_dict': encoder2.state_dict(),
            'opt_g_state_dict': opt_g.state_dict(),
            'opt_d_state_dict': opt_d.state_dict(),
            'opt_en2_state_dict': opt_en2.state_dict(),
            'best_loss_d': best_loss_d,
        }, checkpoint_file)
        print(f"💾 Checkpoint saved at epoch {epoch} with Loss D: {avg_loss_d:.4f}")

    # === Early Stopping ===
    if avg_loss_d < 0.3:
        print(f"⛔ Early stopping at epoch {epoch} as Discriminator loss reached {avg_loss_d:.4f}")
        break

    # === Logging ===
    if epoch % 10 == 0:
        print(f"📘 Epoch {epoch}: Loss D = {avg_loss_d:.4f}, Loss G = {avg_loss_g:.4f}, Recon = {avg_loss_rec:.4f}, Latent = {avg_loss_enc:.4f}")
        


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y=losses_d, mode='lines', name='Discriminator Loss'))
fig.add_trace(go.Scatter(y=losses_g, mode='lines', name='Generator Loss'))
fig.add_trace(go.Scatter(y=losses_rec, mode='lines', name='Reconstruction Loss'))
fig.add_trace(go.Scatter(y=losses_enc, mode='lines', name='Latent Loss'))

fig.update_layout(
    title='Training Losses Over Epochs',
    xaxis_title='Epoch',
    yaxis_title='Loss',
    legend_title='Loss Type',
    template='plotly_white',

)

fig.show()


In [ ]:
# Discriminator Loss
fig_d = go.Figure()
fig_d.add_trace(go.Scatter(y=losses_d, mode='lines', name='Discriminator Loss'))
fig_d.update_layout(title='Discriminator Loss', xaxis_title='Epoch', yaxis_title='Loss', template='plotly_dark')
fig_d.show()

# Generator Loss
fig_g = go.Figure()
fig_g.add_trace(go.Scatter(y=losses_g, mode='lines', name='Generator Loss'))
fig_g.update_layout(title='Generator Loss', xaxis_title='Epoch', yaxis_title='Loss', template='plotly_dark')
fig_g.show()

# Reconstruction Loss
fig_rec = go.Figure()
fig_rec.add_trace(go.Scatter(y=losses_rec, mode='lines', name='Reconstruction Loss'))
fig_rec.update_layout(title='Reconstruction Loss', xaxis_title='Epoch', yaxis_title='Loss', template='plotly_dark')
fig_rec.show()

# Latent Loss
fig_enc = go.Figure()
fig_enc.add_trace(go.Scatter(y=losses_enc, mode='lines', name='Latent Loss'))
fig_enc.update_layout(title='Latent Loss', xaxis_title='Epoch', yaxis_title='Loss', template='plotly_dark')
fig_enc.show()


In [ ]:
checkpoint = torch.load('checkpoints/ganomaly_checkpoint.pth')

encoder.load_state_dict(checkpoint['encoder_state_dict'])
decoder.load_state_dict(checkpoint['decoder_state_dict'])
discriminator.load_state_dict(checkpoint['discriminator_state_dict'])
encoder2.load_state_dict(checkpoint['encoder2_state_dict'])

encoder.eval()
decoder.eval()
discriminator.eval()
encoder2.eval()


In [ ]:
def compute_scores(data_tensor):
    with torch.no_grad():
        z = encoder(data_tensor)
        x_recon = decoder(z)
        z_recon = encoder2(x_recon)

        # Discriminator output (lower is more fake)
        disc_score = discriminator(data_tensor).squeeze()  # shape: (N,)

        # Reconstruction error (L1)
        recon_score = torch.mean(torch.abs(data_tensor - x_recon), dim=1)

        # Latent error (L2)
        latent_score = torch.mean((z - z_recon) ** 2, dim=1)

        # Combined score
        total_score = recon_score + latent_score

        return disc_score.cpu().numpy(), recon_score.cpu().numpy(), latent_score.cpu().numpy(), total_score.cpu().numpy()


In [ ]:
scores_train = compute_scores(train_data_tensor)
scores_test = compute_scores(test_data_tensor)
scores_anomaly = compute_scores(anomaly_data_tensor)



In [ ]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame({
    'Score': list(scores_train[0]) + list(scores_test[0]) + list(scores_anomaly[0]),
    'Set': (['Train'] * len(scores_train[0])) +
           (['Test'] * len(scores_test[0])) +
           (['Anomaly'] * len(scores_anomaly[0]))
})

fig = px.box(df, x='Set', y='Score', title='Discriminator Scores', template='plotly_dark')
fig.show()


In [ ]:
df = pd.DataFrame({
    'Score': list(scores_train[1]) + list(scores_test[1]) + list(scores_anomaly[1]),
    'Set': (['Train'] * len(scores_train[1])) +
           (['Test'] * len(scores_test[1])) +
           (['Anomaly'] * len(scores_anomaly[1]))
})

fig = px.box(df, x='Set', y='Score', title='Reconstruction (L1) Scores', template='plotly_dark')
fig.show()


In [ ]:
df = pd.DataFrame({
    'Score': list(scores_train[2]) + list(scores_test[2]) + list(scores_anomaly[2]),
    'Set': (['Train'] * len(scores_train[2])) +
           (['Test'] * len(scores_test[2])) +
           (['Anomaly'] * len(scores_anomaly[2]))
})

fig = px.box(df, x='Set', y='Score', title='Latent (L2) Scores', template='plotly_dark')
fig.show()


In [ ]:
df = pd.DataFrame({
    'Score': list(scores_train[3]) + list(scores_test[3]) + list(scores_anomaly[3]),
    'Set': (['Train'] * len(scores_train[3])) +
           (['Test'] * len(scores_test[3])) +
           (['Anomaly'] * len(scores_anomaly[3]))
})

fig = px.box(df, x='Set', y='Score', title='Combined (L1 + L2) Scores', template='plotly_dark')
fig.show()


In [ ]:
import numpy as np
from sklearn.metrics import (
    roc_curve, precision_recall_curve, auc,
    accuracy_score, f1_score, precision_score, recall_score
)
import plotly.graph_objects as go

# Replace these with your actual scores
scores_train_2 = np.array(scores_train[2])
scores_test_2 = np.array(scores_test[2])
scores_anomaly_2 = np.array(scores_anomaly[2])

# Test data is normal (label 0), anomaly is (label 1)
y_true = np.concatenate([np.zeros_like(scores_test_2), np.ones_like(scores_anomaly_2)])
y_scores = np.concatenate([scores_test_2, scores_anomaly_2])

# === ROC Curve ===
fpr, tpr, _ = roc_curve(y_true, y_scores)
roc_auc = auc(fpr, tpr)

roc_fig = go.Figure()
roc_fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC Curve'))
roc_fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(dash='dash')))
roc_fig.update_layout(title=f'ROC Curve (AUC = {roc_auc:.4f})',
                      xaxis_title='False Positive Rate',
                      yaxis_title='True Positive Rate')

# === PR Curve ===
precision, recall, _ = precision_recall_curve(y_true, y_scores)
pr_auc = auc(recall, precision)

pr_fig = go.Figure()
pr_fig.add_trace(go.Scatter(x=recall, y=precision, mode='lines', name='PR Curve'))
pr_fig.update_layout(title=f'Precision-Recall Curve (AUC = {pr_auc:.4f})',
                     xaxis_title='Recall',
                     yaxis_title='Precision')

# === Best threshold based on F1 Score ===
thresholds = np.linspace(0, 1, 200)
best_f1 = 0
best_threshold = 0

for t in thresholds:
    y_pred = (y_scores >= t).astype(int)
    f1 = f1_score(y_true, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

# Final metrics
final_preds = (y_scores >= best_threshold).astype(int)
final_acc = accuracy_score(y_true, final_preds)
final_prec = precision_score(y_true, final_preds)
final_rec = recall_score(y_true, final_preds)

# Show plots
roc_fig.show()
pr_fig.show()

# Print best metrics
print(f"Best Threshold: {best_threshold:.4f}")
print(f"Accuracy: {final_acc:.4f}")
print(f"Precision: {final_prec:.4f}")
print(f"Recall: {final_rec:.4f}")
print(f"F1 Score: {best_f1:.4f}")


In [ ]:
import numpy as np
from sklearn.metrics import (
    roc_curve, precision_recall_curve, auc,
    accuracy_score, f1_score, precision_score, recall_score
)
import plotly.graph_objects as go

# Replace these with your actual scores
scores_train_2 = np.array(scores_train[3])
scores_test_2 = np.array(scores_test[3])
scores_anomaly_2 = np.array(scores_anomaly[3])

# Test data is normal (label 0), anomaly is (label 1)
y_true = np.concatenate([np.zeros_like(scores_test_2), np.ones_like(scores_anomaly_2)])
y_scores = np.concatenate([scores_test_2, scores_anomaly_2])

# === ROC Curve ===
fpr, tpr, _ = roc_curve(y_true, y_scores)
roc_auc = auc(fpr, tpr)

roc_fig = go.Figure()
roc_fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC Curve'))
roc_fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(dash='dash')))
roc_fig.update_layout(title=f'ROC Curve (AUC = {roc_auc:.4f})',
                      xaxis_title='False Positive Rate',
                      yaxis_title='True Positive Rate')

# === PR Curve ===
precision, recall, _ = precision_recall_curve(y_true, y_scores)
pr_auc = auc(recall, precision)

pr_fig = go.Figure()
pr_fig.add_trace(go.Scatter(x=recall, y=precision, mode='lines', name='PR Curve'))
pr_fig.update_layout(title=f'Precision-Recall Curve (AUC = {pr_auc:.4f})',
                     xaxis_title='Recall',
                     yaxis_title='Precision')

# === Best threshold based on F1 Score ===
thresholds = np.linspace(0, 1, 200)
best_f1 = 0
best_threshold = 0

for t in thresholds:
    y_pred = (y_scores >= t).astype(int)
    f1 = f1_score(y_true, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

# Final metrics
final_preds = (y_scores >= best_threshold).astype(int)
final_acc = accuracy_score(y_true, final_preds)
final_prec = precision_score(y_true, final_preds)
final_rec = recall_score(y_true, final_preds)

# Show plots
roc_fig.show()
pr_fig.show()

# Print best metrics
print(f"Best Threshold: {best_threshold:.4f}")
print(f"Accuracy: {final_acc:.4f}")
print(f"Precision: {final_prec:.4f}")
print(f"Recall: {final_rec:.4f}")
print(f"F1 Score: {best_f1:.4f}")


In [ ]:
def get_latents(encoder, data_tensor, batch_size=1024):
    latents = []
    encoder.eval()
    with torch.no_grad():
        for i in range(0, data_tensor.size(0), batch_size):
            batch = data_tensor[i:i + batch_size]
            latent = encoder(batch)
            latents.append(latent.cpu().numpy())
    return np.concatenate(latents, axis=0)

latent_train_1 = get_latents(encoder, train_data_tensor)
latent_test_1 = get_latents(encoder, test_data_tensor)
latent_anomaly_1 = get_latents(encoder, anomaly_data_tensor)

latent_train_2 = get_latents(encoder2, train_data_tensor)
latent_test_2 = get_latents(encoder2, test_data_tensor)
latent_anomaly_2 = get_latents(encoder2, anomaly_data_tensor)


In [ ]:
def downsample(arr, n=5000):
    idx = np.random.choice(len(arr), size=min(n, len(arr)), replace=False)
    return arr[idx]

# Choose one encoder’s output to visualize (or concatenate both if you want combined)
latent_train = downsample(latent_train_1)
latent_test = downsample(latent_test_1)
latent_anomaly = downsample(latent_anomaly_1)

X = np.vstack([latent_train, latent_test, latent_anomaly])
labels = (['train'] * len(latent_train) +
          ['test'] * len(latent_test) +
          ['anomaly'] * len(latent_anomaly))


In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, perplexity=30, random_state=5)
X_2d = tsne.fit_transform(X)


In [ ]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame(X_2d, columns=['x', 'y'])
df['label'] = labels

fig = px.scatter(df, x='x', y='y', color='label',
                 title='t-SNE Visualization of Encoder Latent Space')
fig.show()


In [ ]:
tsne = TSNE(n_components=3, perplexity=30, random_state=5)
X_3d = tsne.fit_transform(X)

df_3d = pd.DataFrame(X_3d, columns=['x', 'y', 'z'])
df_3d['label'] = labels

fig_3d = px.scatter_3d(df_3d, x='x', y='y', z='z', color='label',
                       title='3D t-SNE Visualization of Encoder Latent Space')
fig_3d.show()



In [ ]:
# Choose one encoder’s output to visualize (or concatenate both if you want combined)
latent_train_2 = downsample(latent_train_2)
latent_test_2 = downsample(latent_test_2)
latent_anomaly_2 = downsample(latent_anomaly_2)

X = np.vstack([latent_train_2, latent_test_2, latent_anomaly_2])
labels = (['train'] * len(latent_train_2) +
          ['test'] * len(latent_test_2) +
          ['anomaly'] * len(latent_anomaly_2))

tsne = TSNE(n_components=2, perplexity=30, random_state=5)
X_2d = tsne.fit_transform(X)

df = pd.DataFrame(X_2d, columns=['x', 'y'])
df['label'] = labels

fig = px.scatter(df, x='x', y='y', color='label',
                 title='t-SNE Visualization of Encoder Latent Space')
fig.show()


In [ ]:
tsne = TSNE(n_components=3, perplexity=30, random_state=5)
X_3d = tsne.fit_transform(X)


df_3d = pd.DataFrame(X_3d, columns=['x', 'y', 'z'])
df_3d['label'] = labels

fig_3d = px.scatter_3d(df_3d, x='x', y='y', z='z', color='label',
                       title='3D t-SNE Visualization of Encoder Latent Space')
fig_3d.show()